In [1]:
import copy
import lxml.html
from pathlib import Path
import pickle
import re
import time
from tqdm import tqdm

from stanza.server import CoreNLPClient


from projectmir.xmldocument import XMLDocument
from projectmir.functions import find_identifier_definition

In [2]:
# p_path = './data/test_latexml/'
# p = Path(p_path)
# x = list(p.glob('*.html'))
# documents_IDs = [x_.name[:-5] for x_ in x]

# doc = [[]]*len(documents_IDs)
# for i, documents_ID in tqdm(enumerate(documents_IDs)):
#     documents_path = p_path + documents_ID + '.html'
#     doc_ = find_identifier_definition(documents_path)
#     doc[i] = doc_

# save_filename = 'doc' + time.strftime('_%d%b%Y_%H%M%S') + '.pkl'
# with open(save_filename, 'wb') as f:
#     pickle.dump(doc, f)


In [3]:
load_filename = 'doc.pkl'
with open(load_filename, 'rb') as f:
    doc = pickle.load(f)
print(doc[0].path)
print(doc[0].identifiers)

./data/test_latexml/test_sample.html
['w_i', 'T_i', 'Q', 'T', 'w', 'V', 'ρ', 'C', 'U_int', 'H', '\\overset{^}{U}_int', '\\overset{^}{H}', 'd', 't', 'T_ref', '\\overset{^}{H}_ref', '\\overset{^}{H}_i', 'Δ', 'N_F']


In [4]:
doc0 = doc[0]

In [5]:
doc0.sentences

[['The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001.',
  'The inlet and outlet flow rates are equal; thus, MATH0000=MATH0004 and the liquid holdup MATH0005 is constant.'],
 ['The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001.'],
 ['The tank contents are agitated and heated using an electrical heater that provides a heating rate, MATH0002.'],
 ['Perfect mixing: thus, the exit temperature MATH0003 is also the temperature of the tank contents.',
  'Suppose that the liquid in the tank is at a temperature MATH0003 and has an enthalpy, MATH0011.'],
 ['The inlet and outlet flow rates are equal; thus, MATH0000=MATH0004 and the liquid holdup MATH0005 is constant.'],
 ['The inlet and outlet flow rates are equal; thus, MATH0000=MATH0004 and the liquid holdup MATH0005 is constant.'],
 ['The density MATH0006 and heat capacity MATH0007 of the liquid are assumed

In [111]:
# 前処理で，ピリオドを有する単語をピリオドなしの形式に変換しておく．
# この前処理なしで，ピリオド間の文を説明文をゆうする文の候補としても良さそう
# 前処理をしたら候補として文が得られるはずなので，前処理を行う．
# Eq.だったり，Fig.fig.だったりをEquation，Figureのように書き換える．
doc0.text = re.sub('Fig.', 'Figure', doc0.text)


In [112]:
doc0.tagged_sentence_list

[[[('The', 'DT'),
   ('liquid', 'JJ'),
   ('inlet', 'NN'),
   ('stream', 'NN'),
   ('consists', 'VBZ'),
   ('of', 'IN'),
   ('a', 'DT'),
   ('single', 'JJ'),
   ('component', 'NN'),
   ('with', 'IN'),
   ('a', 'DT'),
   ('mass', 'NN'),
   ('flow', 'NN'),
   ('rate', 'NN'),
   ('MATH0000', 'NN'),
   ('and', 'CC'),
   ('an', 'DT'),
   ('inlet', 'JJ'),
   ('temperature', 'NN'),
   ('MATH0001', 'NN'),
   ('.', '.')],
  [('The', 'DT'),
   ('inlet', 'NN'),
   ('and', 'CC'),
   ('outlet', 'NN'),
   ('flow', 'NN'),
   ('rates', 'NNS'),
   ('are', 'VBP'),
   ('equal', 'JJ'),
   (';', ':'),
   ('thus', 'RB'),
   (',', ','),
   ('MATH0000', 'NN'),
   ('=', 'JJ'),
   ('MATH0004', 'NN'),
   ('and', 'CC'),
   ('the', 'DT'),
   ('liquid', 'JJ'),
   ('holdup', 'NN'),
   ('MATH0005', 'NN'),
   ('is', 'VBZ'),
   ('constant', 'JJ'),
   ('.', '.')]],
 [[('The', 'DT'),
   ('liquid', 'JJ'),
   ('inlet', 'NN'),
   ('stream', 'NN'),
   ('consists', 'VBZ'),
   ('of', 'IN'),
   ('a', 'DT'),
   ('single', 'JJ'),

In [95]:
x = [x + '.' for x in doc0.text.split('.') if 'MATH' in x]

In [105]:
x = [re.sub(r'\n', ' ', x.lstrip()) for x in x]

In [108]:
for x_ in x:
    print(x_)

The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001.
The tank contents are agitated and heated using an electrical heater that provides a heating rate, MATH0002.
Perfect mixing: thus, the exit temperature MATH0003 is also the temperature of the tank contents.
The inlet and outlet flow rates are equal; thus, MATH0000=MATH0004 and the liquid holdup MATH0005 is constant.
The density MATH0006 and heat capacity MATH0007 of the liquid are assumed to be constant.
For a pure liquid at low or moderate pressures, the internal energy is approximately equal to the enthalpy, MATH0008≈MATH0009, and MATH0009 depends only on temperature (Sandler, 2006).
Consequently, in the subsequent development, we assume that MATH0008=MATH0009 and MATH0010=MATH0011 where the caret (^) means per unit mass.
As shown in Appendix C, a differential change in temperature MATH0012⁢MATH0003, produces a corresponding change in the internal energy per unit m

In [84]:
re.sub('\n+', ' ', doc0.text.replace('\u2062', ''))

' Process dynamics and control Shota Kato June 8, 2020 1 DYNAMIC MODELS OF REPRESENTATIVE PROCESSES 1.1 Stirred-Tank Heating Process: Constant Holdup Consider the stirred-tank heating system shown in Fig. 2.3. The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001. The tank contents are agitated and heated using an electrical heater that provides a heating rate, MATH0002. A dynamic model will be developed based on the following assumptions: 1.  Perfect mixing: thus, the exit temperature MATH0003 is also the temperature of the tank contents. 2.  The inlet and outlet flow rates are equal; thus, MATH0000=MATH0004 and the liquid holdup MATH0005 is constant. 3.  The density MATH0006 and heat capacity MATH0007 of the liquid are assumed to be constant. Thus, their temperature dependence is neglected. 4.  Heat losses are negligible. In general, dynamic models are based on conservation laws. For this example, it is clear that we s

In [47]:
re.sub('\n+', ' ', doc0.text)

' Process dynamics and control Shota Kato June 8, 2020 1 DYNAMIC MODELS OF REPRESENTATIVE PROCESSES 1.1 Stirred-Tank Heating Process: Constant Holdup Consider the stirred-tank heating system shown in Fig. 2.3. The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001. The tank contents are agitated and heated using an electrical heater that provides a heating rate, MATH0002. A dynamic model will be developed based on the following assumptions: 1.  Perfect mixing: thus, the exit temperature MATH0003 is also the temperature of the tank contents. 2.  The inlet and outlet flow rates are equal; thus, MATH0000=MATH0004 and the liquid holdup MATH0005 is constant. 3.  The density MATH0006 and heat capacity MATH0007 of the liquid are assumed to be constant. Thus, their temperature dependence is neglected. 4.  Heat losses are negligible. In general, dynamic models are based on conservation laws. For this example, it is clear that we s

In [45]:
# T_sentences = re.findall(r'[\.:;](.*?' + 'MATH{:04d}'.format(3) + r'.*?[\.:;])',
#                          re.sub('\n+', ' ', doc0.text))
T_sentences = re.findall(r'^.*(?=MATH{:04d}.format(3)).*$', doc0.text)
a = [s.lstrip() for s in T_sentences]
#if '=' not in s]

In [46]:
a

[]

In [70]:
for identifier, description in zip(doc0.identifiers, doc0.candidate_noun_phrases_list):
    print(identifier)
    for description_ in description:
        for description__ in description_:
            def remove_stopwords(description):
                description_output = description
                stopwords = ['\n', '(', ')', 'NP', 'DT', 'JJ', 'NN', 'PP', 'IN', 'CC']
                for stopword in stopwords:
                    description_output = description_output.replace(stopword, '')
                return description_output
            print(remove_stopwords(description__))
            # print(description__.replace('\n', ''))

    print('\n')

w_i
  The  liquid  inlet  stream
    a  single  component    with            a  mass  flow  rate  MATH0000       and        an  inlet  temperature  MATH0001
  a  single  component
    a  mass  flow  rate  MATH0000   and    an  inlet  temperature  MATH0001
  a  mass  flow  rate  MATH0000
  an  inlet  temperature  MATH0001
  The        inlet     and      outlet  flow S rates
    inlet   and    outlet  flow S rates
  inlet
  outlet  flow S rates
    MATH0000  =  MATH0004   and    the  liquid  holdup  MATH0005
  MATH0000  =  MATH0004
  the  liquid  holdup  MATH0005


T_i
  The  liquid  inlet  stream
    a  single  component    with            a  mass  flow  rate  MATH0000       and        an  inlet  temperature  MATH0001
  a  single  component
    a  mass  flow  rate  MATH0000   and    an  inlet  temperature  MATH0001
  a  mass  flow  rate  MATH0000
  an  inlet  temperature  MATH0001


Q
  The  tank S contents
    an  electrical  heater  SBAR    WH W that    S      VP VBZ provides         

In [160]:
for i, x in enumerate(doc0.identifiers):
    print(i, x)

0 w_i
1 T_i
2 Q
3 T
4 w
5 V
6 ρ
7 C
8 U_int
9 H
10 \overset{^}{U}_int
11 \overset{^}{H}
12 d
13 t
14 T_ref
15 \overset{^}{H}_ref
16 \overset{^}{H}_i
17 Δ
18 N_F


In [5]:
with open('./doc0_text.txt', 'w') as f:
    f.write(doc0.text)
with open('./doc0_body.txt', 'w') as f:
    f.write(doc0.body)

In [60]:
(doc0.candidate_noun_phrases_list[0][0] + doc0.candidate_noun_phrases_list[0][1])

13

In [20]:
with open('./doc0_113130.txt', 'w') as f:
    f.write('\n'.join(doc0.sentences[0]))

In [ ]:
from stanza.server import CoreNLPClient


In [104]:
def extract_noun_phrases(identifiers, sentences):
    candidate_noun_phrases_list = [[]] * len(identifiers)
    pattern = 'NP'
    with CoreNLPClient(
            annotators=['tokenize', 'ssplit', 'pos', 'lemma', 'ner', 'parse', 'depparse', 'coref'],
            timeout=600000, memory='16G') as client:
        for i, sentence in enumerate(sentences):
            if sentence:
                candidate_noun_phrases_list_ = []
                for text in sentence:
                    matches = client.tregex(text, pattern)
                    matches = matches['sentences'][0]
                    candidates = [matches[match]['match'].rstrip()
                                    for match in matches]
                    candidate_noun_phrases_list_ += candidates

                print('##################')
                print(type(candidate_noun_phrases_list_))
                print(candidate_noun_phrases_list_)
                print('##################')

                candidate_noun_phrases_list[i] = candidate_noun_phrases_list_
    return candidate_noun_phrases_list

In [107]:
xx = extract_noun_phrases(doc0.identifiers[:1], doc0.sentences[:1])

Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-c7c45c2a039b4d1b.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
##################
<class 'list'>
['(NP (DT The) (JJ liquid) (NN inlet) (NN stream))', '(NP\n  (NP (DT a) (JJ single) (NN component))\n  (PP (IN with)\n    (NP\n      (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))\n      (CC and)\n      (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))))', '(NP (DT a) (JJ single) (NN component))', '(NP\n  (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))\n  (CC and)\n  (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))', '(NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))', '(NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001))', '(NP (DT The)\n  (NP\n    (NP

In [112]:
print('\n\n'.join(xx[0]))

(NP (DT The) (JJ liquid) (NN inlet) (NN stream))

(NP
  (NP (DT a) (JJ single) (NN component))
  (PP (IN with)
    (NP
      (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))
      (CC and)
      (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))))

(NP (DT a) (JJ single) (NN component))

(NP
  (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))
  (CC and)
  (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))

(NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))

(NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001))

(NP (DT The)
  (NP
    (NP (NN inlet))
    (CC and)
    (NP (NN outlet) (NN flow) (NNS rates))))

(NP
  (NP (NN inlet))
  (CC and)
  (NP (NN outlet) (NN flow) (NNS rates)))

(NP (NN inlet))

(NP (NN outlet) (NN flow) (NNS rates))

(NP
  (NP (NN MATH0000) (JJ =) (NN MATH0004))
  (CC and)
  (NP (DT the) (JJ liquid) (NN holdup) (NN MATH0005)))

(NP (NN MATH0000) (JJ =) (NN MATH0004))

(NP (DT the) (JJ liquid) (NN holdup) (NN MATH0005))


In [135]:
doc0.tagged_sentence_list[0]

[[('The', 'DT'),
  ('liquid', 'JJ'),
  ('inlet', 'NN'),
  ('stream', 'NN'),
  ('consists', 'VBZ'),
  ('of', 'IN'),
  ('a', 'DT'),
  ('single', 'JJ'),
  ('component', 'NN'),
  ('with', 'IN'),
  ('a', 'DT'),
  ('mass', 'NN'),
  ('flow', 'NN'),
  ('rate', 'NN'),
  ('MATH0000', 'NN'),
  ('and', 'CC'),
  ('an', 'DT'),
  ('inlet', 'JJ'),
  ('temperature', 'NN'),
  ('MATH0001', 'NN'),
  ('.', '.')],
 [('The', 'DT'),
  ('inlet', 'NN'),
  ('and', 'CC'),
  ('outlet', 'NN'),
  ('flow', 'NN'),
  ('rates', 'NNS'),
  ('are', 'VBP'),
  ('equal', 'JJ'),
  (';', ':'),
  ('thus', 'RB'),
  (',', ','),
  ('MATH0000', 'NN'),
  ('=', 'JJ'),
  ('MATH0004', 'NN'),
  ('and', 'CC'),
  ('the', 'DT'),
  ('liquid', 'JJ'),
  ('holdup', 'NN'),
  ('MATH0005', 'NN'),
  ('is', 'VBZ'),
  ('constant', 'JJ'),
  ('.', '.')]]

In [121]:
doc0.candidate_noun_phrases_list[0][0]
# -> 最初のidentifierに対するnoun candidate
# candidate にidentifierとの距離を埋め込む．-> candidate.extract_noun_phraseとtagged_sentence_listを比較して，長さを計算する．


['(NP (DT The) (JJ liquid) (NN inlet) (NN stream))',
 '(NP\n  (NP (DT a) (JJ single) (NN component))\n  (PP (IN with)\n    (NP\n      (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))\n      (CC and)\n      (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))))',
 '(NP (DT a) (JJ single) (NN component))',
 '(NP\n  (NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))\n  (CC and)\n  (NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001)))',
 '(NP (DT a) (NN mass) (NN flow) (NN rate) (NN MATH0000))',
 '(NP (DT an) (JJ inlet) (NN temperature) (NN MATH0001))']

In [134]:
# 最初のidentifier[0]が含まれる1つ目の文章[0]の1番目のcandidate[0]
doc0.candidate_noun_phrases_list[0][0][0].replace('NP', '')

'( (DT The) (JJ liquid) (NN inlet) (NN stream))'

In [123]:
doc0.sentences[0][0]

'The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001.'

In [ ]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs